In [ ]:
import numpy as np
import os
import scipy.io as io

In [ ]:
directory_asd = 'shareddata/Project_ASD/clean-trials/Autism/'
directory_typical = 'shareddata/Project_ASD/clean-trials/Neurotypical/'
sub_dir = '/divided/'

In [ ]:
list_dir_asd = list(os.listdir(directory_asd))
list_dir_typical = list(os.listdir(directory_typical))
n_asd = len(list_dir_asd)-1 
n_typical = len(list_dir_typical)-1 
dat_asd = np.zeros((n_asd,2,4), dtype=object)
dat_typical = np.zeros((n_typical,2,4), dtype=object)
i = 0
for folder_name in list_dir_asd:
    if folder_name[0]!='a':
        for j in range(4):
            file_name_c = 'catch-C'+str(j+1)+'.mat'
            file_name_t = 'target-C'+str(j+1)+'.mat'
            load_c = io.loadmat(directory_asd+folder_name
                                        +sub_dir+file_name_c)
            load_t = io.loadmat(directory_asd+folder_name
                                        +sub_dir+file_name_t)
            # catch
            dat_asd[i,0,j] = load_c['data']#\
                            #- load_c['baseline50'][:,np.newaxis]
            # target
            dat_asd[i,1,j] = load_t['data']#\
                            #- load_t['baseline50'][:,np.newaxis]
        i+=1
i = 0
for folder_name in list_dir_typical:
    if folder_name[0]!='a':
        for j in range(4):
            file_name_c = 'catch-C'+str(j+1)+'.mat'
            file_name_t = 'target-C'+str(j+1)+'.mat'
            load_c = io.loadmat(directory_typical+folder_name
                                            +sub_dir+file_name_c)
            load_t = io.loadmat(directory_typical+folder_name
                                            +sub_dir+file_name_t)
            # catch
            dat_typical[i,0,j] = load_c['data']#\
                                #- load_c['baseline50'][:,np.newaxis]
            # target
            dat_typical[i,1,j] = load_t['data']#\
                                #- load_t['baseline50'][:,np.newaxis]
        i+=1

In [ ]:
save_directory = 'shareddata/Project_ASD/'
np.savez(save_directory+'ERPs.npz',
         ERPs_ASD=dat_asd, ERPs_TYP=dat_typical)

In [ ]:
# dat_asd[0,0,0].shape

In [ ]:
# for i in range(4):
#     print(dat_asd[0,0,i]['data'].shape)

In [ ]:
# dat_asd[0,0,i]['baseline100']#.shape

In [ ]:
# OLD

"""
Saving conditions separately and labeling asd vs typical 
groups are here to perform cross-validation by excluding participants
and not trial at random: we like te predict asd or not for individuals.
"""
ERPs = np.zeros((4), dtype=object)
groups = np.zeros((4), dtype=object) 
asd = np.zeros((4), dtype=object)

for i in range(4):
    for j in range(n_asd):
        conds = dat_asd[j]['evConds'][0]==i+1
        if j == 0:
            ERPs[i] = dat_asd[j]['ERPs'][:,:,conds]
            groups[i] = j*np.ones(conds.sum())
            asd[i] = np.ones(conds.sum())
        else:
            ERPs[i] = np.concatenate((ERPs[i],dat_asd[j]['ERPs'][:,:,conds]),
                                     axis=2)
            groups[i] = np.concatenate((groups[i],j*np.ones(conds.sum())))
            asd[i] = np.concatenate((asd[i], np.ones(conds.sum())))
            
    for j in range(n_typical):
        conds = dat_typical[j]['evConds'][0]==i+1
        ERPs[i] = np.concatenate((ERPs[i],dat_typical[j]['ERPs'][:,:,conds]),
                                 axis=2)
        groups[i] = np.concatenate((groups[i],(n_asd+j)*np.ones(conds.sum())))
        asd[i] = np.concatenate((asd[i], np.zeros(conds.sum())))

In [ ]:
save_directory = 'shareddata/3Dmat4ML/'
np.savez(save_directory+'ERPs.npz', ERPs_C0=ERPs[0], ERPs_C1=ERPs[1], ERPs_C2=ERPs[2],
         ERPs_C3=ERPs[3], groups=groups, asd=asd)

In [ ]:
# help(np.savez)